In [1]:
!pip install paramiko boto boto3

You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import aws_helper as a # this python file has a bunch of messy thrown-together
                       # methods using boto3 libraries with amazon and a ssh python interface
                       # called paramiko
import threading
import os

In [4]:
## Instance Parameters

MD_AMI = "ami-02db52e359e89b39d" # this AMI should have up-to-date libwallerlab
                                 # it also needs to have aws client installed and configured with
                                 # credentials
                                 # old ami: "ami-0f6e873188b7105e6"
INSTANCE_TYPE = "p2.xlarge"  # "p2.xlarge" "t2.micro" "r5a.2xlarge"
                               # p2 is GPU instance for deblurring, 
                               # t2 is cheap, good for testing basic stuff
                               # r5 has lots of memory, good for stitching
KEYNAME = "p2xps" # this is the name of the key on Amazon (if you launch an instance manually)
                  # through the console, you will be prompted to create and download keypairs
# KEYPATH = '/home/sarah/.ssh/p2xps.pem' # this is the location of the key on your computer
KEYPATH = os.path.expanduser('~/.ssh/p2xps.pem') # this is the location of the key on your computer

PRICE = '0.4' # for spot instances
num_instances = 1 # the number of instances


In [7]:
## Starting spot requests

instance_ids = []; public_dnses = []
for i in range(num_instances):
    instance_id, public_dns = a.start_instance(MD_AMI, PRICE, KEYNAME, INSTANCE_TYPE)
    print('id:', instance_id)
    print('dns:', public_dns)
    instance_ids.append(instance_id); 
    public_dnses.append(public_dns)
    
    
# TODO current there is a bug in the way instances IDs are collected, so the list might not be accurate 

ClientError: An error occurred (InvalidGroup.NotFound) when calling the RequestSpotInstances operation: The security group 'sg-0f5165083df8e479f' does not exist in VPC 'vpc-4864b730'

In [10]:
## Collecting DNS of all open instances

instance_list = a.get_all_instances()
print(instance_list)
public_dnses = []
for instance in instance_list: # instance_list[0]
    print(instance[0])
    if instance[0] != '': # instance != '':
        public_dnses.append(instance[0])
print(public_dnses)

# TODO sometimes the returned list is buggy

[['ec2-54-244-162-28.us-west-2.compute.amazonaws.com'], ['ec2-54-149-145-243.us-west-2.compute.amazonaws.com']]
ec2-54-244-162-28.us-west-2.compute.amazonaws.com
ec2-54-149-145-243.us-west-2.compute.amazonaws.com
['ec2-54-244-162-28.us-west-2.compute.amazonaws.com', 'ec2-54-149-145-243.us-west-2.compute.amazonaws.com']


In [12]:
# if all else fails, manually collect DNS from ec2 console (under "Instances") -- make sure the 
# region is correct because the console only shows instances in a given region (us-west-2 is my default)

# public_dnses = ['ec2-54-202-61-221.us-west-2.compute.amazonaws.com', 
#                 'ec2-54-212-219-53.us-west-2.compute.amazonaws.com',
#                'ec2-34-208-98-113.us-west-2.compute.amazonaws.com',
#                'ec2-34-222-213-10.us-west-2.compute.amazonaws.com']


In [5]:
# Sending commands to instances

In [5]:
## Show all datasets loaded onto each instance

outlock = threading.Lock()

test_cmd = 'cd ~/datasets; ls' #; sleep 5'

threads = []
for i in range(num_instances): #num_instances):
    t = threading.Thread(target=a.send_command_to_instance, args=(test_cmd, public_dnses[i], KEYPATH))
    t.start()
    threads.append(t)
for t in threads:
    t.join()

res_target_color_coded_raster_motiondeblur_2018_05_22_19_17_45
res_target_color_strobe_raster_motiondeblur_2018_05_22_19_17_18_recovered_[19, 20, 21, 22, 23].npz




In [ ]:
## Copying datasets from s3 to instance

outlock = threading.Lock()

filename = '163c_color_coded_raster_motiondeblur_2018_05_23_08_50_25'
filename = '163a_color_coded_raster_motiondeblur_2018_05_22_21_47_35'
filename = '174d_color_coded_raster_motiondeblur_2018_05_23_16_18_15'

# TODO the below method of string construction for some reason does not work...
copy_file = 'aws s3 cp s3://motiondeblur/ ' + filename + '/ ~/datasets/' + filename + ' --recursive --region us-east-2'

copy_file_163c = 'aws s3 cp s3://motiondeblur/163c_color_coded_raster_motiondeblur_2018_05_23_08_50_25/ ~/datasets/163c_color_coded_raster_motiondeblur_2018_05_23_08_50_25 --recursive --region us-east-2'
copy_file_163a = 'aws s3 cp s3://motiondeblur/163a_color_coded_raster_motiondeblur_2018_05_22_21_47_35/ ~/datasets/163a_color_coded_raster_motiondeblur_2018_05_22_21_47_35 --recursive --region us-east-2'
copy_file_174d = 'aws s3 cp s3://motiondeblur/174d_color_coded_raster_motiondeblur_2018_05_23_16_18_15/ ~/datasets/174d_color_coded_raster_motiondeblur_2018_05_23_16_18_15 --recursive --region us-east-2'


copy_file = 'aws s3 cp s3://motiondeblur/beads_coded_raster_100/ ~/datasets/beads_coded_raster_100 --recursive --region us-east-2'


threads = []
for i in range(num_instances):
    if i in [0,1,2]: # logic about which datasets to which instance
        cmd = 'rm -rf ~/datasets/*; ' + copy_file_163a
    elif i in [3,4,5]:
        cmd = 'rm -rf ~/datasets/*; ' + copy_file_174d
    t = threading.Thread(target=a.send_command_to_instance, args=(cmd, public_dnses[i], KEYPATH))
    t.start()
    threads.append(t)
for t in threads:
    t.join()


In [ ]:
## Running reconstructions on each instance

outlock = threading.Lock()

# note: by default instance open to home directory, so it is necessary to cd into correct directory
change_dir = 'cd /home/ubuntu/libwallerlab/libwallerlab/projects/motiondeblur/experimental_reconstructions; '

# the script run_recons.sh calls the reconstruction python script
reconstruct = ["./run_recons.sh 0 29 coded_163a.conf '--channel 0' > output.txt; ",
               "./run_recons.sh 0 29 coded_163a.conf '--channel 1' > output.txt; ",
               "./run_recons.sh 0 29 coded_163a.conf '--channel 2' > output.txt; ",
               "./run_recons.sh 0 29 coded_174d.conf '--channel 0' > output.txt; ",
               "./run_recons.sh 0 29 coded_174d.conf '--channel 1' > output.txt; ",
               "./run_recons.sh 0 29 coded_174d.conf '--channel 2' > output.txt; "]

threads_recon = []
for i in range(num_instances):
    cmd = change_dir + reconstruct[i]
    t = threading.Thread(target=a.send_command_to_instance, args=(cmd, public_dnses[i], KEYPATH))
    t.start()
    threads_recon.append(t)

In [ ]:
# Calling t.join() here waits until all reconstructions are done -- usually I don't run this cell

for t in threads_recon:
    t.join()

In [ ]:
## Checking the status of recons on each instance based on the file output.txt

# prints output of reconstruction script
# outputs update after each frame

outlock = threading.Lock()

change_dir = 'cd /home/ubuntu/libwallerlab/libwallerlab/projects/motiondeblur/experimental_reconstructions; '
status = 'cat output.txt'

threads = []
for i in range(num_instances):
    cmd = change_dir + status
    t = threading.Thread(target=a.send_command_to_instance, args=(cmd, public_dnses[i], KEYPATH))
    t.start()
    threads.append(t)
for t in threads:
    t.join()


In [6]:
## Copying reconstructed strips to instance

public_dns = public_dns[0]
# copy_files = 'aws s3 cp s3://motiondeblur/reconstructions/ ./ --recursive --exclude "*" --include "163c_color_coded_raster_motiondeblur_2018_05_23_08_50_25_strip*_channel=1*" --region us-east-2'
copy_files = 'aws s3 cp s3://motiondeblur/reconstructions/ ./ --recursive --exclude "*" --include "beads_*" --region us-east-2'
cmd = 'cd ~/reconstructions; ' + copy_files # + 'ls ~/datasets'
a.send_command_to_instance(cmd, public_dns, KEYPATH)

NameError: name 'public_dns' is not defined

In [11]:
## Copying reconstructed strips to instance

# copy_files = 'aws s3 cp s3://motiondeblur/reconstructions/ ./ --recursive --exclude "*" --include "163c_color_coded_raster_motiondeblur_2018_05_23_08_50_25_strip*_channel=1*" --region us-east-2'
copy_files = 'aws s3 cp s3://motiondeblur/reconstructions/ ./ --recursive --exclude "*" --include "beads_*" --region us-east-2'
cmd = 'cd ~/datasets; ' + copy_files # + 'ls ~/datasets'
import subprocess
subprocess.run([copy_files, ""])

FileNotFoundError: [Errno 2] No such file or directory: 'aws s3 cp s3://motiondeblur/reconstructions/ ./ --recursive --exclude "*" --include "beads_*" --region us-east-2': 'aws s3 cp s3://motiondeblur/reconstructions/ ./ --recursive --exclude "*" --include "beads_*" --region us-east-2'

In [ ]:
## Stitching all strips on instance

# I usually do this via jupyter notebook batch_segment_stitching_aws.ipynb:
# 1) manually ssh into machine with `ssh -i PATH_TO_KEY ubuntu@PUBLIC_DNS`
# 2) get jupyter notebook up and running in a screen: `screen -S notebook` then `jupyter notebook` 
#    usually from libwallerlab directory so I can edit any necessary code through the notebook interfact
#    the screen can be detached with ctrl+a d, and reattached with `screen -r notebook`
#    allowing you to do multiple things at once on the instance. Note the token that appears when the 
#    notebook server starts running -- you'll need this to log in
# 3) on your own computer, start an ssh pipe: 
#    `ssh -i PATH_TO_KEY -N -L 8080:localhost:8888 ubuntu@PUBLIC_DNS
#    [this assumes the jupyter notebook on the machine is at port 8888]
# 4) go to localhost:8080 and enter the token
